In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install nltk

In [ ]:
pip install -U gensim

In [ ]:
# Load library

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, matthews_corrcoef, roc_curve, auc
import pickle
from keras.models import load_model
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle as pk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

#from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

#from keras.utils.np_utils import to_categorical
import re
from __future__ import print_function
from keras.models import Sequential
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras import backend as K
from __future__ import print_function
import _pickle as cPickle

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers import Convolution1D, MaxPooling1D

from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
from keras.models import model_from_json
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn import metrics

import random
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

In [ ]:
def twoTupleDic():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        for j in AA_list_sort:
          for jj in AA_list_sort:

            AA_dict[i+j+jj] = numm
            numm += 1
    return AA_dict

In [ ]:
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence

In [ ]:
path_data = "/content/drive/MyDrive/CLW_SUMO/Data/"
path_result = "/content/drive/MyDrive/CLW_SUMO/Results/"
path_model = "/content/drive/MyDrive/CLW_SUMO/Model/"


In [ ]:
resutl_file="WCL_Sumo.txt"
name_result_test ="Independent_test.txt"

In [ ]:
# Tokenizer train data input Word2vec
data_token = []
for i in df_train['n_gram']:
   data_token.append(i.split())
print(data_token[:2])

[['SWS', 'WSP', 'SPK', 'PKI', 'KIK', 'IKR', 'KRE', 'REK', 'EKS', 'KSS', 'SSV'], ['HVV', 'VVF', 'VFG', 'FGQ', 'GQV', 'QVV', 'VVE', 'VEG', 'EGL', 'GLN', 'LNV']]


In [ ]:
# load test data
file_test ="Test_Sumo.csv"
df_test =pd.read_csv(path_data+file_test)
text_test =[] #PTMsequend kmer
for i in df_test['Sequence']:
  temp = ProSentence(i,k) # Biểu diễn dữ liệu đầu vào thành token Kmer
  text_test.append(temp)
df_test['n_gram'] =text_test
df_test

from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle as cPickle
import pandas as pd
max_features =300

word_index1 = twoTupleDic() # Tham chiếu texts thành word_index1  trong bộ từ điển
test_sequences = []
for each in text_test:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    test_sequences.append(each_index_list)
# Tokenizer train data input Word2vec
data_token = []
for i in df_test['k_mer']:
   data_token.append(i.split())
# Len of the K_mer[1]
len(data_token[1])
MAX_SEQUENCE_LENGTH = len(data_token[1])
print(MAX_SEQUENCE_LENGTH)
print(data_token[:2])
Xtest = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytest = np.array(df_test['Label'])

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt

# --- Load mô hình ---
model = load_model("./CLW_SUMO.h5")
# --- Dự đoán ---
y_probs = model.predict(Xtest)
y_preds = (y_probs > 0.5).astype(int).flatten()
y_true = ytest.flatten()

# --- Tính các chỉ số ---
acc = accuracy_score(y_true, y_preds)
prec = precision_score(y_true, y_preds)
rec = recall_score(y_true, y_preds)
f1 = f1_score(y_true, y_preds)
auc_score = roc_auc_score(y_true, y_probs)

cm = confusion_matrix(y_true, y_preds)
tn, fp, fn, tp = cm.ravel()

# --- In kết quả ---
print("===== Evaluation on Independent Test Set =====")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-Score : {f1:.4f}")
print(f"AUC-ROC  : {auc_score:.4f}")
print(f"Confusion Matrix: TP={tp}, FP={fp}, TN={tn}, FN={fn}")

# --- Lưu kết quả ra Excel (tùy chọn) ---
df_metrics = pd.DataFrame([{
    "Accuracy": acc,
    "Precision": prec,
    "Recall": rec,
    "F1-score": f1,
    "AUC-ROC": auc_score,
    "TP": tp,
    "FP": fp,
    "TN": tn,
    "FN": fn
}])

df_metrics.to_excel("CLW_SUMO_ID.xlsx", index=False)
print("\nĐã lưu kết quả vào 'CLW_SUMO_ID.xlsx'.")

# --- Vẽ đường cong ROC ---
fpr, tpr, _ = roc_curve(y_true, y_probs)
plt.figure(figsize=(6, 5))
